## Vaccination
Illustration of the rationale for and process of applying the vaccination extension to the base model.

In [ ]:
# Uncomment the following commands to install in Colab
# Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment
# and then proceed to the following cells

# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.7b2 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch finalise-notebooks
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
import pandas as pd
pd.options.plotting.backend = 'plotly'
from aust_covid.inputs import get_base_vacc_data, get_ifrs
from aust_covid.vaccination import get_vacc_data_masks, add_derived_data_to_vacc, get_full_vacc_props
from aust_covid.plotting import plot_full_vacc, plot_program_coverage, plot_immune_props, plot_vacc_implementation
from emutools.tex import DummyTexDoc, StandardTexDoc
from inputs.constants import INPUTS_PATH, SUPPLEMENT_PATH
from emutools.parameters import load_param_info
from aust_covid.utils import add_image_to_doc
from aust_covid.model import build_model
from aust_covid.extra_text import add_vaccination_blurb_to_tex

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'vaccination', 'Vaccination', 'austcovid')

In [ ]:
vacc_df = get_base_vacc_data()
masks = get_vacc_data_masks(vacc_df)
vacc_df, lagged_df = add_derived_data_to_vacc(vacc_df)
full_prop_df = get_full_vacc_props(vacc_df, masks['age 16+, 2+ doses'])

In [ ]:
full_vacc_fig = plot_full_vacc(masks['age 16+, 2+ doses'], vacc_df, full_prop_df)
full_vacc_fig

In [ ]:
caption = "Second (`full') dose roll-out by age group. Number of persons receiving second dose (upper panel) and " \
    'proportion of population having received second dose (lower panel). ' \
    'Age groups coloured from cyan (12 to 15 years-old) to purple (95+ years-old). '
add_image_to_doc(full_vacc_fig.update_layout(showlegend=False), 'full_vacc', 'svg', 'Full vaccination coverage', app_doc, 'Vaccination', caption=caption)

In [ ]:
coverage_by_program = plot_program_coverage({m: masks[m] for m in ['age 16+, 3+ doses', 'age 16+, 4+ doses', 'age 12-15, 2+ doses', 'age 5-11, 2+ doses']}, vacc_df)
add_image_to_doc(coverage_by_program, 'program_coverage', 'svg', 'Vaccination coverage by subsequent programs', app_doc, 'Vaccination')
coverage_by_program

In [ ]:
caption = 'Booster program for persons aged 16 and above (red line), primary vaccination course for persons aged 5 to 11.'
implement_fig = plot_vacc_implementation(vacc_df)
add_image_to_doc(implement_fig.update_layout(showlegend=False), 'vacc_implement', 'svg', 'Vaccination implementation.', app_doc, 'Vaccination', caption=caption)
implement_fig

In [ ]:
param_info = load_param_info()
param_info['value'].update(get_ifrs(DummyTexDoc()))
parameters = param_info['value'].to_dict()
vacc_model = build_model(DummyTexDoc(), param_info['abbreviations'], vacc_sens=True)

In [ ]:
vacc_model.run(parameters=parameters)

In [ ]:
vacc_distribution = plot_immune_props(vacc_model, vacc_df, lagged_df)
caption = 'Comparison of modelled and empiric vaccination coverage'
vacc_distribution

In [ ]:
caption = 'Reported vaccination coverage by program (dashed black line), and lagged vaccination coverage (dotted black line). ' \
    'Coloured areas represent distribution of population by vaccination status under vaccination extension: not yet vaccinated under program (green), ' \
    'recently vaccinated under program (red), and vaccinated under program but protective effect lost (blue).'
add_image_to_doc(vacc_distribution.update_layout(showlegend=False), 'vaccination_distribution', 'svg', 'Comparison of reported to modelled vaccination status distribution. ', app_doc, 'Vaccination', caption=caption)
add_vaccination_blurb_to_tex(app_doc)
app_doc.write_doc()

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex vaccination
    ! biber vaccination
    ! pdflatex vaccination
    ! pdflatex vaccination